In [3]:
import duckdb
import pandas as pd

##TODO: Falta bajar el dataset directamente de Kaggle
# import kagglehub
# from kagglehub import KaggleDatasetAdapter

# # Load the dataset from KaggleHub
# df = kagglehub.load_dataset(
#     KaggleDatasetAdapter.PANDAS,
#     "alejandroczernikier/properati-argentina-dataset",
#     file_path="/data/raw"
# )




con = duckdb.connect("entrenamiento.duckdb")

# Usamos un SELECT sobre read_csv_auto()
con.execute("""
COPY (
    SELECT * FROM read_csv_auto('data/RAW/entrenamiento.csv')
) TO 'data/parquet/entrenamiento.parquet' (FORMAT PARQUET)
""")

#guardarlo en duckdb

con = duckdb.connect("data/DB/entrenamiento.duckdb")

# Crear tabla desde el archivo Parquet
con.execute("""
    CREATE TABLE entrenamiento AS
    SELECT * FROM 'data/parquet/entrenamiento.parquet'
""")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
print(con.execute("PRAGMA table_info(entrenamiento)").fetchdf())

    cid             name     type  notnull dflt_value     pk
0     0               id   BIGINT    False       None  False
1     1          ad_type  VARCHAR    False       None  False
2     2       start_date     DATE    False       None  False
3     3         end_date     DATE    False       None  False
4     4       created_on     DATE    False       None  False
5     5              lat   DOUBLE    False       None  False
6     6              lon   DOUBLE    False       None  False
7     7               l1  VARCHAR    False       None  False
8     8               l2  VARCHAR    False       None  False
9     9               l3  VARCHAR    False       None  False
10   10               l4  VARCHAR    False       None  False
11   11               l5  VARCHAR    False       None  False
12   12               l6  VARCHAR    False       None  False
13   13            rooms   DOUBLE    False       None  False
14   14         bedrooms   DOUBLE    False       None  False
15   15        bathrooms